In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import tree
import os
import subprocess
import statistics
import warnings



In [2]:
data = pd.read_csv('iris.csv', header=None)
X = data.iloc[: , :4]
y = pd.Series(map(str, data.iloc[: , 4]))

## Compilación optimizada

In [3]:
os.system('cargo rustc --release -- -C opt-level=3 -C target-cpu=native -C debuginfo=0')

   --> src/dtree.rs:234:42
    |
234 |                     range_threshold.push((pss.0 + i as f32 * delta));
    |                                          ^                        ^
    |
    = note: `#[warn(unused_parens)]` on by default
help: remove these parentheses
    |
234 -                     range_threshold.push((pss.0 + i as f32 * delta));
234 +                     range_threshold.push(pss.0 + i as f32 * delta);
    |

 --> src/dtree.rs:3:5
  |
3 | use rayon::prelude::*;
  |     ^^^^^^^^^^^^^^
  |
  = note: `#[warn(unused_imports)]` on by default

 --> src/dtree.rs:7:5
  |
7 | use std::io::prelude::*;
  |     ^^^^^^^^^^^^^^^^

 --> src/dtree.rs:1:5
  |
1 | use rand::prelude::*;
  |     ^^^^^^^^^^^^^

 --> src/main.rs:3:5
  |
3 | use rand::prelude::*;
  |     ^^^^^^^^^^^^^

   --> src/dtree.rs:196:21
    |
196 |             let mut c = 0;
    |                     ^
    |
    = note: consider using `_c` instead
    = note: `#[warn(unused_variables)]` on by default

  --> src/

0

## Ejecución SK LEARN

In [4]:
ns = range(1000, 300001, 1000)

In [5]:
tree_outputs = []
raw_outputs = []
for row in ns:
    X_ = X.iloc[:row, :]
    y_ = y.iloc[:row]
    clf = tree.DecisionTreeClassifier(criterion='gini')
    t = %timeit -n 1 -r 5 -o clf.fit(X_, y_)
    
    tree_outputs.append(np.mean(t.all_runs)*1000)
    raw_outputs.append(t)

5.33 ms ± 2.03 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
4.69 ms ± 1.23 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
5.16 ms ± 223 µs per loop (mean ± std. dev. of 5 runs, 1 loop each)
6.34 ms ± 159 µs per loop (mean ± std. dev. of 5 runs, 1 loop each)
8.19 ms ± 316 µs per loop (mean ± std. dev. of 5 runs, 1 loop each)
9.83 ms ± 309 µs per loop (mean ± std. dev. of 5 runs, 1 loop each)
12.1 ms ± 1.56 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
12.3 ms ± 877 µs per loop (mean ± std. dev. of 5 runs, 1 loop each)
13.6 ms ± 493 µs per loop (mean ± std. dev. of 5 runs, 1 loop each)
18.5 ms ± 1.93 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
20.1 ms ± 4.13 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
17.5 ms ± 109 µs per loop (mean ± std. dev. of 5 runs, 1 loop each)
20.7 ms ± 2.09 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
20.5 ms ± 161 µs per loop (mean ± std. dev. of 5 runs, 1 loop each)
22.3 ms ± 811 µs per loop (mean ± std. dev

228 ms ± 4.84 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
229 ms ± 3.36 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
233 ms ± 3.7 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
239 ms ± 8.74 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
239 ms ± 5.25 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
242 ms ± 12.1 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
240 ms ± 3.35 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
240 ms ± 3.67 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
249 ms ± 4.47 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
248 ms ± 3.75 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
257 ms ± 3.7 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
254 ms ± 4.49 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
257 ms ± 3.89 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
253 ms ± 3.53 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
258 ms ± 3.18 ms per loop (mean ± std. dev. of 5 r

KeyboardInterrupt: 

## Tiempos rust para IRIS 300000

In [6]:
warnings.filterwarnings("ignore")

# Define a list of input values
n = 5
# Initialize an empty list to store the outputs
outputs = []

# Loop over the input values and execute the command

for row in ns:
    inner_t = []

    for i in range(n):
        # Execute the command and capture the output
        output = subprocess.check_output(
            [f'./target/release/decision-tree {row} 4 iris'], shell=True, text=True)
        print(output)
        # Append the output to the list of outputs
        
        inner_t.append(float(output.strip('ms\n')))
        # outputs.append(output)

    outputs.append(np.mean(inner_t))

    # Print the outputs

outputs_ = outputs.copy()

for i in range(len(outputs_)):
    if outputs_[i] < 10:
        outputs_[i] = outputs_[i]*1000
        


258.874273ms

289.794778ms

271.883804ms



KeyboardInterrupt: 

In [ ]:


plt.plot(ns, tree_outputs, label='sk-learn')
plt.plot(ns, outputs_, label="rust optimizado")
plt.xlabel("Tamaño del conjunto")
plt.ylabel("Tiempo de entraniento [ms]")
plt.legend()


## Prueba dataset simple (iris)

In [ ]:
inner_t = []

for i in range(50):
    # Execute the command and capture the output
    output = subprocess.check_output(
        [f'./target/release/decision-tree 150 4 iris'], shell=True, text=True)

    # Append the output to the list of outputs

    inner_t.append(float(output.strip('ms\n')))
    # outputs.append(output)

inner_t